In [ ]:
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import os
import math
import subprocess

from scipy.optimize import curve_fit
from scipy.special import gamma
from scipy.special import gammaincc

In [ ]:
def defaultParam():
    """
    """
    defaultParam = {
        'pixelWidth': 10.,
        'pixelThickness': 5.,
        'pitch': 55.,
        'meshStandoff': 100.,
        'meshThickness': 4.,
        'holeRadius': 16.,
        'cathodeHeight': 400.,
        'thicknessSiO2': 5.,
        'meshVoltage': -140.,
        'cathodeVoltage': -180.,
        'numFieldLine': 11,
        'transparencyLimit': .01,
        'numAvalanche': 1000,
        'avalancheLimit': 200,
        'gasCompAr': 80.,
        'gasCompCO2': 20.,
        'penningR': 0.51,
        'penningLambda': 0.
    }
    return defaultParam
    
def readParam():
    """
    """
    filename='runControl'
    param = {}
    
    try:
        with open(filename, 'r') as file:
            for line in file:
                line = line.strip()  # Remove leading/trailing whitespace
                if line.startswith('/') or not line:  # Skip comments and empty lines
                    continue
                # Split the line at the '='
                parts = line.split('=', 1)
                if len(parts) == 2:
                    key, value = parts[0].strip(), parts[1].strip()
                    value = value.rstrip(';') # Remove trailing semicolon
                    param[key] = value
                else:
                    print(f"Skipping malformed line: {line}")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return {}
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return {}

    return param


def writeRunControl(param=None):
    """
    """
    filename='runControl'

    if param is None or not param:
        print(f"No parameters to write to {filename}.")
        return

    try:
        with open(filename, 'r') as file:
            oldLines = file.readlines()  # Read all lines of the file
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

    newLines = []
    for line in oldLines:
        line = line.strip()
        if line.startswith('/') or not line:
            newLines.append(line + '\n')  # Keep comments and empty lines
            continue

        parts = line.split('=', 1)
        if len(parts) == 2:
            key = parts[0].strip()
            if key in param:
                newLines.append(f"{key} = {param[key]};\n")  # Update value
            else:
                newLines.append(line + '\n') #keep original line
        else:
            newLines.append(line + '\n')  # Keep non-parameter lines

    try:
        with open(filename, 'w') as file:
            file.writelines(newLines)
    except Exception as e:
        print(f"An error occurred while writing to {filename}: {e}")
    
    return


def writeSIF(param=None):
    """
    """
    filename = 'FIMS.sif'

    if param is None or not param:
        print(f"Invalid parameters.")
        return None
    
    try:
        with open(filename, 'r') as file:
            sifLines = file.readlines()  # Read all lines of the file

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return
    
    #Rewrite lines 149 and 155 - TODO - make these not explicit linenos
    sifLines[149] = f'  Potential = {param['cathodeVoltage']}\n'
    sifLines[155] = f'  Potential = {param['meshVoltage']}\n'
        
    try:
        with open(filename, 'w') as file:
            file.writelines(sifLines)
    except Exception as e:
        print(f"An error occurred while writing to {filename}: {e}")
        return
        
    return


def writeParam(param=None):
    """
    """
    if param is None or not param:
        print(f"No parameters to write.")
        return    

    writeRunControl(param)
    writeSIF(param)
    
    return


def resetParam():
    """
    """
    defaultParams = defaultParam()
    writeParam(defaultParams)

    print('Parameters have been reset.')
    
    return defaultParams
    

In [ ]:
def calcPotentials(param=None, driftField=1, amplificationField=80):
    """

    E field = Voltage / Distance -> V = E*d
    """

    if param is None or not param:
        print(f"Invalid parameters.")
        return None
        
    convertEField = 0.1 # 1 kV/cm = 0.1 V/micron
    
    # Calculate the voltage required to achieve desired amplification field
    meshDistance = param['meshStandoff'] - param['meshThickness']/2. #micron
    meshVoltage = amplificationField*convertEField*meshDistance

    # Calculate for drift field
    cathodeDistance = param['cathodeHeight'] - param['meshThickness']/2. #micron
    cathodeVoltage = driftField*convertEField*cathodeDistance + meshVoltage

    potentials = {
        'Mesh': -meshVoltage,
        'Cathode': -cathodeVoltage
    }
    
    return potentials

def findNewPotentials(param=None, driftField=1, amplificationField=80):
    """
    """

    if param is None or not param:
        print(f"Invalid parameters.")
        return None

    newPotentials = calcPotentials(param, driftField, amplificationField)

    param['cathodeVoltage'] = newPotentials['Cathode']
    param['meshVoltage'] = newPotentials['Mesh']

    # Write files
    writeParam(param)
    
    return newPotentials  

In [ ]:
def runGmsh():
    subprocess.run(['Programs/gmsh', 'Geometry/FIMS.txt', '-3'])
    return

def runElmer():
    subprocess.run(['ElmerGrid', '14', '2', 'Geometry/FIMS.msh', '-out', 'Geometry', '-autoclean'])
    subprocess.run(['ElmerSolver', 'Geometry/FIMS.sif'])
    return

def runGarfield():
    subprocess.run(['~/Work/garfield/garfieldpp/electronAvalanche/fims/DataGeneration/FIMS_SIM/build/runAvalanche'])
    return

def runSimulation():
    runGmsh()
    runElmer()
    runGarfield()
    return

In [ ]:
def getRunNumber():
    """
    """
    filename='runNo'

    try:
        with open(filename, 'r') as file:
            content = file.read().strip()
            runNo = int(content)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except ValueError:
        print(f"Error: Invalid number format in '{filename}")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return
    
    return runNo

In [ ]:
def readRootTrees(runNumber: int) -> dict:
    """
    Reads all trees from a ROOT file and returns them as a dictionary.
    The keys are the tree names and the values are pandas DataFrames.

    Args:
        runNumber (int): The run number of the ROOT file.

    Returns:
        dict: A dictionary where keys are tree names (str) and
              values are pandas DataFrames. Returns an empty dictionary
              if the file cannot be opened or contains no trees.
    """

    dataFilePath = 'Data/'
    dataFile = f'sim.{runNumber}.root'
    fullFilePath = os.path.join(dataFilePath, dataFile)
    
    try:
        with uproot.open(fullFilePath) as rootFile:
            dataframes = {}
            for treeName in rootFile.keys():
                if isinstance(rootFile[treeName], uproot.behaviors.TTree.TTree):
                    tree = rootFile[treeName]
                    try:
                        df = tree.arrays(library='pd')
                        dataframes[treeName] = df
                    except Exception as e:
                        print(f"Error reading tree '{treeName}': {e}")
            return dataframes
    except Exception as e:
        print(f"Error opening or reading ROOT file '{fullFilePath}': {e}")
        return {}


def getAvalancheTree(numNumber: int):
    """
    """
    simData = readRootTrees(runNumber)

    for key in simData:
        if key.startswith('metaData'):
            metaData = simData[key]
        if key.startswith('avalancheData'):
            avalancheData = simData[key]

    return metaData, avalancheData

In [ ]:
def myPolya(n, nBar, theta):
    return (1/nBar)*np.power(theta+1, theta+1)/gamma(theta+1)*np.power(n/nBar, theta)*np.exp(-(n/nBar)*(theta+1))
    
def fitPolya(avalancheData, metaData, binWidth=1):
    """
    """
    bins = np.arange(avalancheData['Total Electrons'].min(), 
                     avalancheData['Total Electrons'].max()+1, 
                     binWidth)
    binCenters = bins[:-1] + binWidth/2.

    counts, _ = np.histogram(avalancheData['Total Electrons'], 
                             bins=bins)
    prob = counts/len(avalancheData['Total Electrons'])/binWidth

    # Get error
    countErr = np.where(counts == 0, 1, np.sqrt(counts))
    probErr = countErr/len(avalancheData['Total Electrons'])/binWidth

    # Set initial fitting parameters with some bounds
    initial = [avalancheData['Total Electrons'].mean(), 1.]
    bounds = ([1, 0], [metaData['Avalanche Limit'].iloc[0], 10])

    popt, pcov = curve_fit(myPolya, binCenters, prob, p0=initial, bounds=bounds, sigma=probErr)
    perr = np.sqrt(np.diag(pcov))

    fitPolyaParam = {
        'nBar': popt[0],
        'nBarErr': perr[0],
        'theta': popt[1],
        'thetaErr': perr[1]
    }

    return fitPolyaParam

def myPolyaEfficiency(nBar, theta, threshold=10):
    s = theta+1
    x = s*threshold/nBar
    return gammaincc(s, x)

In [ ]:
def getPolyaEfficiency(runNumber, threshold=10):
    """
    """
    metaData, avalancheData = getAvalancheTree(numNumber)

    fitParam = fitPolya(avalancheData, metaData)

    efficiency = myPolyaEfficiency(fitParam['nBar'], fitParam['theta'], threshold)

    print(f'Run {runNumber}: '
          +r'$\bar{n}$'
          +f'={fitParam['nBar']:.0f}, '
          +r'$\theta$'+f'={fitParam['theta']:.4f}')

    return efficiency    

In [ ]:
def varyRadius(rMin, rMax, numSteps):
    """
    """
    param = readParam()

    runNo = getRunNumber()
    print(f'Varying the radius. Initial run number: {runNo}')

    radii = np.linspace(rMin, rMax, numSteps)
    efficiency = {}
    
    for inRad in radii:
        param['holeRadius'] = inRad
        findNewPotentials(param)        

        runSimulation()

        runNo = getRunNumber()
        efficiency += [getPolyaEfficiency(runNo)]

    runNo = getRunNumber()
    print(f'Done varying the radius. Final run number: {runNo-1}')

    param = resetParam()
    
    return radii, efficiency